In [64]:
!pip3 install indic-nlp-library

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [65]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer
import torch 
from indicnlp.tokenize import indic_tokenize

In [66]:
data =pd.read_csv("/kaggle/input/sample-dataset/Sentence pairs in English-Hindi - 2025-02-11.tsv",sep="\t",header=None,names=["SrcSentId","SrcSent","DataSentId","DstSent"])

In [67]:
data

,SrcSentId,SrcSent,DataSentId,DstSent
0,1282,Muiriel is 20 now.,485968,म्यूरियल अब बीस साल की हो गई है।
1,1282,Muiriel is 20 now.,2060319,म्यूरियल अब बीस साल की है।
2,1294,Education in this world disappoints me.,485564,मैं इस दुनिया में शिक्षा पर बहुत निराश हूँ।
3,1302,That won't happen.,2060320,वैसा नहीं होगा।
4,1308,I miss you.,2060321,मुझें तुम्हारी याद आ रही है।
...,...,...,...,...
13177,12886228,Do you have some curry and some rice?,12886225,क्या आपके पास सब्ज़ी-चावल है?
13178,12886229,Do you have curry and some rice?,12886225,क्या आपके पास सब्ज़ी-चावल है?
13179,12886230,Do you have any curry with rice?,12886225,क्या आपके पास सब्ज़ी-चावल है?
13180,12886231,Do you have curry with rice?,12886225,क्या आपके पास सब्ज़ी-चावल है?


In [68]:
data.head()

,SrcSentId,SrcSent,DataSentId,DstSent
0,1282,Muiriel is 20 now.,485968,म्यूरियल अब बीस साल की हो गई है।
1,1282,Muiriel is 20 now.,2060319,म्यूरियल अब बीस साल की है।
2,1294,Education in this world disappoints me.,485564,मैं इस दुनिया में शिक्षा पर बहुत निराश हूँ।
3,1302,That won't happen.,2060320,वैसा नहीं होगा।
4,1308,I miss you.,2060321,मुझें तुम्हारी याद आ रही है।


In [69]:
data.shape

(13182, 4)

In [70]:
data.drop(labels=[data.columns[0],data.columns[2]],axis=1,inplace=True)

In [71]:
data.head()

,SrcSent,DstSent
0,Muiriel is 20 now.,म्यूरियल अब बीस साल की हो गई है।
1,Muiriel is 20 now.,म्यूरियल अब बीस साल की है।
2,Education in this world disappoints me.,मैं इस दुनिया में शिक्षा पर बहुत निराश हूँ।
3,That won't happen.,वैसा नहीं होगा।
4,I miss you.,मुझें तुम्हारी याद आ रही है।


In [72]:
src_sent_tokenizer=AutoTokenizer.from_pretrained("google-T5/T5-base")

In [73]:
data["SrcSent"]= data["SrcSent"].apply(lambda x: src_sent_tokenizer.tokenize(x))

In [74]:
data.head()

,SrcSent,DstSent
0,"[▁Mu, i, riel, ▁is, ▁20, ▁now, .]",म्यूरियल अब बीस साल की हो गई है।
1,"[▁Mu, i, riel, ▁is, ▁20, ▁now, .]",म्यूरियल अब बीस साल की है।
2,"[▁Education, ▁in, ▁this, ▁world, ▁disappoint, ...",मैं इस दुनिया में शिक्षा पर बहुत निराश हूँ।
3,"[▁That, ▁won, ', t, ▁happen, .]",वैसा नहीं होगा।
4,"[▁I, ▁miss, ▁you, .]",मुझें तुम्हारी याद आ रही है।


In [75]:
data["DstSent"]= data["DstSent"].apply(lambda x: indic_tokenize.trivial_tokenize(x,lang="hi"))

In [76]:
data.head()

,SrcSent,DstSent
0,"[▁Mu, i, riel, ▁is, ▁20, ▁now, .]","[म्यूरियल, अब, बीस, साल, की, हो, गई, है, ।]"
1,"[▁Mu, i, riel, ▁is, ▁20, ▁now, .]","[म्यूरियल, अब, बीस, साल, की, है, ।]"
2,"[▁Education, ▁in, ▁this, ▁world, ▁disappoint, ...","[मैं, इस, दुनिया, में, शिक्षा, पर, बहुत, निराश..."
3,"[▁That, ▁won, ', t, ▁happen, .]","[वैसा, नहीं, होगा, ।]"
4,"[▁I, ▁miss, ▁you, .]","[मुझें, तुम्हारी, याद, आ, रही, है, ।]"


In [77]:
data["SrcSent"]= data["SrcSent"].apply(src_sent_tokenizer.convert_tokens_to_ids)

In [78]:
data.head()

,SrcSent,DstSent
0,"[4159, 23, 14018, 19, 460, 230, 5]","[म्यूरियल, अब, बीस, साल, की, हो, गई, है, ।]"
1,"[4159, 23, 14018, 19, 460, 230, 5]","[म्यूरियल, अब, बीस, साल, की, है, ।]"
2,"[2855, 16, 48, 296, 26963, 7, 140, 5]","[मैं, इस, दुनिया, में, शिक्षा, पर, बहुत, निराश..."
3,"[466, 751, 31, 17, 1837, 5]","[वैसा, नहीं, होगा, ।]"
4,"[27, 3041, 25, 5]","[मुझें, तुम्हारी, याद, आ, रही, है, ।]"


In [79]:
Vs=src_sent_tokenizer.get_vocab()

In [80]:
len(Vs)

32100

In [81]:
hindi_vocab=set()
for tokenized_hindi_sent in data["DstSent"]:
    hindi_vocab.update(tokenized_hindi_sent)

In [82]:
data.head()

,SrcSent,DstSent
0,"[4159, 23, 14018, 19, 460, 230, 5]","[म्यूरियल, अब, बीस, साल, की, हो, गई, है, ।]"
1,"[4159, 23, 14018, 19, 460, 230, 5]","[म्यूरियल, अब, बीस, साल, की, है, ।]"
2,"[2855, 16, 48, 296, 26963, 7, 140, 5]","[मैं, इस, दुनिया, में, शिक्षा, पर, बहुत, निराश..."
3,"[466, 751, 31, 17, 1837, 5]","[वैसा, नहीं, होगा, ।]"
4,"[27, 3041, 25, 5]","[मुझें, तुम्हारी, याद, आ, रही, है, ।]"


In [83]:
Vd = dict()
for idx,token in enumerate(hindi_vocab):
    Vd[token]=idx+3
Vd["<PAD>"]=0
Vd["<SOS>"]=1
Vd["<EOS>"]=2

In [84]:
len(Vd)

7072

In [85]:
print(type(data["DstSent"].iloc[0]))


<class 'list'>


In [86]:
def convert_hindi_token_to_ids(hindi_sent):
    return [Vd[token] for token in hindi_sent]

In [87]:
data["DstSent"]=data["DstSent"].apply(lambda x:convert_hindi_token_to_ids(x))

In [88]:
data.head()

,SrcSent,DstSent
0,"[4159, 23, 14018, 19, 460, 230, 5]","[532, 6278, 2250, 4413, 5553, 6468, 4014, 3663..."
1,"[4159, 23, 14018, 19, 460, 230, 5]","[532, 6278, 2250, 4413, 5553, 3663, 316]"
2,"[2855, 16, 48, 296, 26963, 7, 140, 5]","[1080, 2229, 1521, 5382, 6133, 4034, 5090, 345..."
3,"[466, 751, 31, 17, 1837, 5]","[1084, 6306, 4958, 316]"
4,"[27, 3041, 25, 5]","[6849, 2808, 359, 3863, 2896, 3663, 316]"


In [89]:
def insert_sos_token_id(hindi_token_sent_ids):
    return [1]+hindi_token_sent_ids

In [90]:
data["DstSentInput"]=data["DstSent"].apply(lambda x:insert_sos_token_id(x))
data.head()

,SrcSent,DstSent,DstSentInput
0,"[4159, 23, 14018, 19, 460, 230, 5]","[532, 6278, 2250, 4413, 5553, 6468, 4014, 3663...","[1, 532, 6278, 2250, 4413, 5553, 6468, 4014, 3..."
1,"[4159, 23, 14018, 19, 460, 230, 5]","[532, 6278, 2250, 4413, 5553, 3663, 316]","[1, 532, 6278, 2250, 4413, 5553, 3663, 316]"
2,"[2855, 16, 48, 296, 26963, 7, 140, 5]","[1080, 2229, 1521, 5382, 6133, 4034, 5090, 345...","[1, 1080, 2229, 1521, 5382, 6133, 4034, 5090, ..."
3,"[466, 751, 31, 17, 1837, 5]","[1084, 6306, 4958, 316]","[1, 1084, 6306, 4958, 316]"
4,"[27, 3041, 25, 5]","[6849, 2808, 359, 3863, 2896, 3663, 316]","[1, 6849, 2808, 359, 3863, 2896, 3663, 316]"


In [91]:
def insert_eos_token_id(hindi_token_sent_ids):
    return hindi_token_sent_ids+[2]

In [92]:
data["DstSentLabel"]=data["DstSent"].apply(lambda x:insert_eos_token_id(x))
data.head()

,SrcSent,DstSent,DstSentInput,DstSentLabel
0,"[4159, 23, 14018, 19, 460, 230, 5]","[532, 6278, 2250, 4413, 5553, 6468, 4014, 3663...","[1, 532, 6278, 2250, 4413, 5553, 6468, 4014, 3...","[532, 6278, 2250, 4413, 5553, 6468, 4014, 3663..."
1,"[4159, 23, 14018, 19, 460, 230, 5]","[532, 6278, 2250, 4413, 5553, 3663, 316]","[1, 532, 6278, 2250, 4413, 5553, 3663, 316]","[532, 6278, 2250, 4413, 5553, 3663, 316, 2]"
2,"[2855, 16, 48, 296, 26963, 7, 140, 5]","[1080, 2229, 1521, 5382, 6133, 4034, 5090, 345...","[1, 1080, 2229, 1521, 5382, 6133, 4034, 5090, ...","[1080, 2229, 1521, 5382, 6133, 4034, 5090, 345..."
3,"[466, 751, 31, 17, 1837, 5]","[1084, 6306, 4958, 316]","[1, 1084, 6306, 4958, 316]","[1084, 6306, 4958, 316, 2]"
4,"[27, 3041, 25, 5]","[6849, 2808, 359, 3863, 2896, 3663, 316]","[1, 6849, 2808, 359, 3863, 2896, 3663, 316]","[6849, 2808, 359, 3863, 2896, 3663, 316, 2]"


In [93]:
data.drop(labels=[data.columns[1]],axis=1,inplace=True)

In [94]:
data.head()

,SrcSent,DstSentInput,DstSentLabel
0,"[4159, 23, 14018, 19, 460, 230, 5]","[1, 532, 6278, 2250, 4413, 5553, 6468, 4014, 3...","[532, 6278, 2250, 4413, 5553, 6468, 4014, 3663..."
1,"[4159, 23, 14018, 19, 460, 230, 5]","[1, 532, 6278, 2250, 4413, 5553, 3663, 316]","[532, 6278, 2250, 4413, 5553, 3663, 316, 2]"
2,"[2855, 16, 48, 296, 26963, 7, 140, 5]","[1, 1080, 2229, 1521, 5382, 6133, 4034, 5090, ...","[1080, 2229, 1521, 5382, 6133, 4034, 5090, 345..."
3,"[466, 751, 31, 17, 1837, 5]","[1, 1084, 6306, 4958, 316]","[1084, 6306, 4958, 316, 2]"
4,"[27, 3041, 25, 5]","[1, 6849, 2808, 359, 3863, 2896, 3663, 316]","[6849, 2808, 359, 3863, 2896, 3663, 316, 2]"


In [95]:
X=list(data["SrcSent"])
y_input=list(data["DstSentInput"])
y_label=list(data["DstSentLabel"])

In [96]:
X_tensor=[torch.tensor(tokenized_eng_sent_ids) for tokenized_eng_sent_ids in X]

In [97]:
Y_input_tensor= [torch.tensor(tokenized_hindi_sent_ids) for tokenized_hindi_sent_ids in y_input]

In [98]:
Y_label_tensor=[torch.tensor(tokenized_hindi_sent_ids) for tokenized_hindi_sent_ids in y_label]

In [99]:
X_padded= torch.nn.utils.rnn.pad_sequence(X_tensor,batch_first=True)


In [100]:
Y_padded_input= torch.nn.utils.rnn.pad_sequence(Y_input_tensor,batch_first=True)
Y_padded_label= torch.nn.utils.rnn.pad_sequence(Y_label_tensor,batch_first=True)

In [101]:
Ns=X_padded.shape[1]
Nd=Y_padded_label.shape[1]

In [102]:
Ns

68

In [103]:
Nd

68

In [144]:
class Encoder(torch.nn.Module):

    def __init__(self,src_lang_vocab_size,word_embedding_dim):
        super(Encoder,self).__init__()
        self.first_embedding_layer = torch.nn.Embedding(num_embeddings=src_lang_vocab_size,
                                                       embedding_dim=word_embedding_dim)
        self.second_lstm_layer = torch.nn.LSTM(input_size=word_embedding_dim,
                                               hidden_size=word_embedding_dim,
                                              batch_first=True)

    def forward(self,X_padded_mini_batch):

        first_embedding_layer_out = self.first_embedding_layer(X_padded_mini_batch)
        encoder_output, (final_encoder_output,final_cell_state) = self.second_lstm_layer(first_embedding_layer_out)

        return encoder_output, (final_encoder_output,final_cell_state)
        
        
        
    

In [146]:
class Decoder(torch.nn.Module):

    def __init__(self,dst_lang_vocab_size,word_embedding_dim):
        super(Decoder,self).__init__()

        self.first_embedding_layer = torch.nn.Embedding(num_embeddings=dst_lang_vocab_size,
                                                       embedding_dim=word_embedding_dim)
        self.second_lstm_layer = torch.nn.LSTM(input_size=word_embedding_dim,
                                               hidden_size=word_embedding_dim,
                                              batch_first=True)
        self.prediction_layer = torch.nn.Linear(in_features=word_embedding_dim,out_features=dst_lang_vocab_size)
        self.prediction_layer_activation = torch.nn.Softmax(dim=2)

    def forward(self,Y_padded_input_mini_batch,final_encoder_output,final_cell_state):

        first_embedding_layer_out = self.first_embedding_layer(Y_padded_input_mini_batch)
        decoder_lstm_layer_out, (final_decoder_lstm_layer_out, final_cell_state) = self.second_lstm_layer(first_embedding_layer_out,
                                                                                                         (final_encoder_output,
                                                                                                          final_cell_state))
        prediction = self.prediction_layer_activation(self.prediction_layer(decoder_lstm_layer_out))
        
        return prediction, (final_decoder_lstm_layer_out, final_cell_state)

In [147]:
class Seq2SeqEncDec(torch.nn.Module):

    def __init__(self,src_lang_vocab_size,dst_lang_vocab_size,word_embedding_dim):
        super(Seq2SeqEncDec,self).__init__()

        self.encoder = Encoder(src_lang_vocab_size,word_embedding_dim)
        self.decoder = Decoder(dst_lang_vocab_size,word_embedding_dim)

    def forward(self,X_padded_mini_batch,Y_padded_input_mini_batch):

        encoder_output, (final_encoder_output,final_cell_state) = self.encoder(X_padded_mini_batch)
        y_hat_mini_batch = self.decoder(Y_padded_input_mini_batch,
                                        final_encoder_output,final_cell_state)

        return y_hat_mini_batch
        

In [148]:
X_padded_train= X_padded[0:13000]
Y_padded_input_train=Y_padded_input[0:13000]
Y_padded_label_train=Y_padded_label[0:13000]

X_padded_test=X_padded[13000:]
Y_padded_input_test=Y_padded_input[13000:]
Y_padded_label_test=Y_padded_label[13000:]

In [149]:
network = Seq2SeqEncDec(len(Vs),len(Vd),128).to(device)
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=0)
optimizer = torch.optim.Adam(network.parameters())
num_epochs = 25
mb_size = 65

for epoch in range(num_epochs):
    for i in range(X_padded_train.shape[0]//mb_size):

        X_train_mb = X_padded_train[i*mb_size:(i+1)*mb_size]
        Y_input_mb = Y_padded_input_train[i*mb_size:(i+1)*mb_size]
        Y_label_mb = Y_padded_label_train[i*mb_size:(i+1)*mb_size]
        Y_label_mb = Y_label_mb.reshape(Y_label_mb.shape[0]*Y_label_mb.shape[1],)
        
        X_train_mb, Y_input_mb, Y_label_mb = X_train_mb.to(device), Y_input_mb.to(device), Y_label_mb.to(device)

        y_hat_train_mb = network(X_train_mb,Y_input_mb)
        y_hat_train_mb = y_hat_train_mb[0]
        y_hat_train_mb = y_hat_train_mb.reshape(y_hat_train_mb.shape[0]*y_hat_train_mb.shape[1],
                                                y_hat_train_mb.shape[2])

        loss_fn_value = loss_fn(y_hat_train_mb,Y_label_mb)

        loss_fn_value.backward()
        torch.nn.utils.clip_grad_norm_(network.parameters(),max_norm=1.0)
        optimizer.step()
        optimizer.zero_grad()

        print("Epoch # {}, Time Step # {}, Loss Value = {}".format(epoch,i,loss_fn_value))
        
        


Epoch # 0, Time Step # 0, Loss Value = 8.863899230957031
Epoch # 0, Time Step # 1, Loss Value = 8.86389446258545
Epoch # 0, Time Step # 2, Loss Value = 8.863893508911133
Epoch # 0, Time Step # 3, Loss Value = 8.86388874053955
Epoch # 0, Time Step # 4, Loss Value = 8.863883972167969
Epoch # 0, Time Step # 5, Loss Value = 8.863877296447754
Epoch # 0, Time Step # 6, Loss Value = 8.863875389099121
Epoch # 0, Time Step # 7, Loss Value = 8.863865852355957
Epoch # 0, Time Step # 8, Loss Value = 8.863862991333008
Epoch # 0, Time Step # 9, Loss Value = 8.863846778869629
Epoch # 0, Time Step # 10, Loss Value = 8.863844871520996
Epoch # 0, Time Step # 11, Loss Value = 8.8638277053833
Epoch # 0, Time Step # 12, Loss Value = 8.863788604736328
Epoch # 0, Time Step # 13, Loss Value = 8.863789558410645
Epoch # 0, Time Step # 14, Loss Value = 8.86374282836914
Epoch # 0, Time Step # 15, Loss Value = 8.863709449768066
Epoch # 0, Time Step # 16, Loss Value = 8.863638877868652
Epoch # 0, Time Step # 17, Lo

In [151]:
torch.save(network.state_dict(), "model.pth")